In [30]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [31]:
!apt-get install -y mysql-server mysql-client

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
mysql-client is already the newest version (8.0.37-0ubuntu0.22.04.3).
mysql-server is already the newest version (8.0.37-0ubuntu0.22.04.3).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [32]:
!service mysql start

 * Starting MySQL database server mysqld
   ...done.


In [33]:
!mysql -e "CREATE DATABASE testdb;"
!mysql -e "CREATE USER 'testuser'@'localhost' IDENTIFIED BY 'testpassword';"
!mysql -e "GRANT ALL PRIVILEGES ON testdb.* TO 'testuser'@'localhost';"
!mysql -e "FLUSH PRIVILEGES;"

ERROR 1007 (HY000) at line 1: Can't create database 'testdb'; database exists
ERROR 1396 (HY000) at line 1: Operation CREATE USER failed for 'testuser'@'localhost'


In [34]:
pip install mysql-connector-python

In [35]:
pip install PyMuPDF

In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from io import BytesIO
import mysql.connector
import fitz

In [37]:
# Configuración de la conexión a MySQL
mydb = mysql.connector.connect(
    host="localhost",
    user="testuser",
    password="testpassword",
    database="testdb"
)


mycursor = mydb.cursor()

# Verificar si la tabla existe
mycursor.execute('DROP TABLE IF EXISTS normativa')

# Verificar si la tabla existe
mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = 'normativa'
        AND table_schema = 'testdb'
    """)
if mycursor.fetchone()[0] == 0:

# Crear la tabla si no existe
    mycursor.execute("""
        CREATE TABLE normativa (
          id int unsigned NOT NULL AUTO_INCREMENT,
          ciudad varchar(100) DEFAULT NULL,
          date date DEFAULT NULL,
          titulo varchar(255) DEFAULT NULL,
          grupo varchar(255) DEFAULT NULL,
          subgrupo varchar(255) DEFAULT NULL,
          url varchar(500) DEFAULT NULL,
          content longtext,
          PRIMARY KEY (id)
          )
          ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;
        """)

    print("Tabla 'normativa' creada.")
else:
    print("La tabla 'normativa' ya existe.")

Tabla 'normativa' creada.


In [38]:
# Definir parámetros base
base_url = 'https://www.aytoburgos.es/buscador?formDate=1718900805374&ddmstructurekey=85278&q=&p_auth=FCK8w2yG&delta=129'
ciudad = 'Burgos'
fecha = datetime.today().strftime('%Y-%m-%d')

headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
        }

response = requests.get(base_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Crear lista para extraer los enlaces de la página
filtered_urls_base = {}

for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    if 'https://www.aytoburgos.es/-/' in href:
        text = a_tag.text.strip()
        filtered_urls_base[text] = href

In [39]:
print(filtered_urls_base)

{'Ordenanza para la Supresión de Barreras Arquitectónicas': 'https://www.aytoburgos.es/-/ordenanza-para-la-supresion-de-barreras-arquitectonicas?p_l_back_url=%2Fbuscador%3FformDate%3D1718900805374%26ddmstructurekey%3D85278%26q%3D%26p_auth%3DFCK8w2yG%26delta%3D129', 'Reglamento del Centro de Creación Musical de Burgos': 'https://www.aytoburgos.es/-/reglamento-del-centro-de-creacion-musical-de-burgos?p_l_back_url=%2Fbuscador%3FformDate%3D1718900805374%26ddmstructurekey%3D85278%26q%3D%26p_auth%3DFCK8w2yG%26delta%3D129', 'Estatutos de la Gerencia Municipal de Servicios Sociales, Juventud e Igualdad de Oportunidades.': 'https://www.aytoburgos.es/-/estatutos-de-la-gerencia-municipal-de-servicios-sociales-juventud-e-igualdad-de-oportunidades.?p_l_back_url=%2Fbuscador%3FformDate%3D1718900805374%26ddmstructurekey%3D85278%26q%3D%26p_auth%3DFCK8w2yG%26delta%3D129', 'III PLAN MUNICIPAL DE INFANCIA, ADOLESCENCIA Y FAMILIA 2020-2024': 'https://www.aytoburgos.es/-/iii-plan-municipal-de-infancia-adole

In [40]:
# Lista para acumular resultados de todas las páginas
filtered_urls = []

for title, base_url in filtered_urls_base.items():
    print(f"Título: {title}, Enlace: {base_url}")

    response = requests.get(base_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            if '/documents/' in href:
                # Construct the full URL if the href is relative
                if not href.startswith('http'):
                    href = requests.compat.urljoin(base_url, href)
                print(f"Downloading PDF from: {href}")
                pdf_response = requests.get(href, headers=headers)
                if pdf_response.status_code == 200 and 'application/pdf' in pdf_response.headers.get('Content-Type', ''):
                    pdf_data = BytesIO(pdf_response.content)
                    doc = fitz.open(stream=pdf_data, filetype="pdf")
                    content = ""
                    for page in doc:
                        content += page.get_text()
                else:
                    print(f"Error al descargar el archivo PDF: {href}")

                filtered_urls.append({
                      'ciudad': ciudad,
                      'date': fecha,
                      'grupo':  '',
                      'subgrupo': '',
                      'titulo': title,
                      'url': base_url,
                      'content': content
                  })
    else:
        print(f"Error al acceder a la página: {base_url}")


# Crear DataFrame con los resultados acumulados
df = pd.DataFrame(filtered_urls)

Título: Ordenanza para la Supresión de Barreras Arquitectónicas, Enlace: https://www.aytoburgos.es/-/ordenanza-para-la-supresion-de-barreras-arquitectonicas?p_l_back_url=%2Fbuscador%3FformDate%3D1718900805374%26ddmstructurekey%3D85278%26q%3D%26p_auth%3DFCK8w2yG%26delta%3D129
Título: Reglamento del Centro de Creación Musical de Burgos, Enlace: https://www.aytoburgos.es/-/reglamento-del-centro-de-creacion-musical-de-burgos?p_l_back_url=%2Fbuscador%3FformDate%3D1718900805374%26ddmstructurekey%3D85278%26q%3D%26p_auth%3DFCK8w2yG%26delta%3D129
Título: Estatutos de la Gerencia Municipal de Servicios Sociales, Juventud e Igualdad de Oportunidades., Enlace: https://www.aytoburgos.es/-/estatutos-de-la-gerencia-municipal-de-servicios-sociales-juventud-e-igualdad-de-oportunidades.?p_l_back_url=%2Fbuscador%3FformDate%3D1718900805374%26ddmstructurekey%3D85278%26q%3D%26p_auth%3DFCK8w2yG%26delta%3D129
Título: III PLAN MUNICIPAL DE INFANCIA, ADOLESCENCIA Y FAMILIA 2020-2024, Enlace: https://www.aytobur

In [41]:
df.shape

(225, 7)

In [42]:
# Insertar los datos en la tabla de MySQL
for index, row in df.iterrows():
    sql = "INSERT INTO normativa (ciudad, date, titulo, grupo, subgrupo, url, content) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    val = (row['ciudad'], row['date'], row['titulo'], row['grupo'], row['subgrupo'], row['url'], row['content'])
    mycursor.execute(sql, val)

mydb.commit()

print(mycursor.rowcount, "registro(s) insertado(s).")

# Consultar los datos de la tabla
mycursor.execute('SELECT * FROM normativa')
rows = mycursor.fetchall()

for row in rows:
    print(row)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [43]:
import os
import tarfile

# Obtener la carpeta de documentos del usuario
directorio_actual = os.getcwd()

# Nombre del archivo donde guardarás el script SQL
archivo_sql = os.path.join(directorio_actual, 'dump_normativa.sql')

# Escribir los comandos INSERT en el archivo
try:
    with open(archivo_sql, 'w', encoding='utf-8') as archivo:
        for row in rows:
            insert_statement = f"INSERT INTO normativa (ciudad, date, titulo, grupo, subgrupo, url, content) VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}', '{row[5]}', '{row[6]}');\n"
            archivo.write(insert_statement)
    print(f"Datos guardados en {archivo_sql}")
except IOError as e:
    print(f"Error al escribir en el archivo {archivo_sql}: {e}")

# Verificar si el archivo se ha creado correctamente
if os.path.exists(archivo_sql):
    print(f"El archivo {archivo_sql} existe.")
else:
    print(f"El archivo {archivo_sql} no se ha creado.")

# Crear un archivo tar.gz y agregar el archivo SQL
nombre_tar = os.path.join(directorio_actual, 'dump_normativa.tar.gz')
try:
    with tarfile.open(nombre_tar, "w:gz") as tar:
        tar.add(archivo_sql, arcname=os.path.basename(archivo_sql))
    print(f"Archivo {nombre_tar} creado correctamente.")
except Exception as e:
    print(f"Error al crear el archivo tar.gz: {e}")

# Verificar si el archivo tar.gz se ha creado correctamente
if os.path.exists(nombre_tar):
    print(f"El archivo {nombre_tar} existe.")
else:
    print(f"El archivo {nombre_tar} no se ha creado.")


Datos guardados en /content/dump_normativa.sql
El archivo /content/dump_normativa.sql existe.
Archivo /content/dump_normativa.tar.gz creado correctamente.
El archivo /content/dump_normativa.tar.gz existe.
